![satisfaction.png](satisfaction.png)

# <font color='black'>Projeto:</font> <font color='blue'>Prevendo o Nível de Satisfação dos Clientes do Santander.</font>

## <font color='black'>Data:</font> <font color='blue'>julho, 2020</font>

## <font color='black'>Cientista de Dados:</font> <font color='blue'>Walter Trevisan</font>

## 06- Tests (`Step 04`)

Nesta etapa, vamos aplicar o melhor modelo preditivo (***`XGBoost Classifier`***) no dataset de **`testes`** da competição, na plataforma **`Kaggle`**, e depois vamos submeter o resultado para verificar o score na própria competição.

Portanto, nesta etapa, vamos realizar as seguintes atividades:

1. Carregar o *dataset* com os dados de teste: **`test.csv`**;


2. Carregar objeto *importante* criado na fase de *Análise Exploratória*, ou seja, **`03-Explore-The-Data (Step-05)`**;


3. Carregar os objetos *importantes* criados na fase de *Pré-processamento*, ou seja, **`04-Preprocessing`**;


4. Carregar os objetos *importantes* criados na fase de *Machine Learning*, ou seja, **`05-Machine-Learning (Step-03)`**;


5. Preparar o *dataset* de teste (**`test.csv`**) para relizarmos os testes com o nosso melhor modelo preditivo;


6. Realizar os testes com o melhor modelo preditivo (**`XGBC`**);


7. Criar e salvar o arquivo que será submetido na competição da plataforma `Kaggle`, ou seja: **`sample_submission.csv`**;


8. Verificar o **`score`** e a posição no **`leaderboard`**, na plataforma `Kaggle`.

## Setup

In [1]:
# As novas versões do Pandas e Matplotlib trazem diversas mensagens de aviso ao desenvolvedor.
# Então, vamos desativar essas mensagens.
import sys # O pacote "sys" permite manipulações com o sistema operacional:
import os  # Operation System (Packages and Functions)
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Common imports:
import numpy as np    # NumPy
import pandas as pd   # Pandas

# Machine Learning imports
from xgboost import XGBClassifier

# Flag para fazer as previsões nos dados de teste:
# Se "True", faz as previsões utilizando a função.
# Se "False", faz a leitura das previsões (objetos) que foram salvas.
PREDICT=True

# Definindo o diretório raiz (Root) onde serão armazenados todas as informações
# (Imagens, Objetos, Dados, Modelos de ML, etc...) do projeto.
# Diretório Raiz (Root) do Projeto:
PROJECT_ROOT_DIR = "."

# Path: onde ficarão armazenados os "Objetos" (Estrututras de Dados) relacionados ao Projeto:
PROJECT_OBJ_PATH = os.path.join(PROJECT_ROOT_DIR, "Objects")
# Criando o diretório, se ele não existir:
os.makedirs(PROJECT_OBJ_PATH, exist_ok=True)

# Path: onde ficarão armazenados os "Modelos" (Machine Learning) relacionados ao Projeto:
PROJECT_ML_PATH = os.path.join(PROJECT_ROOT_DIR, "Models")
# Criando o diretório, se ele não existir:
os.makedirs(PROJECT_ML_PATH, exist_ok=True)

# Path: onde ficará armazenado o arquivo que será submetido na competição (kaggle):
PROJECT_OUTPUT_PATH = os.path.join(PROJECT_ROOT_DIR, "Output")
# Criando o diretório, se ele não existir:
os.makedirs(PROJECT_OUTPUT_PATH, exist_ok=True)

# Path: onde ficarão armazenados os "datasets" (arquivos "csv") e os "Objetos" (Data Frames) do Projeto:
PROJECT_DATA_PATH = os.path.join(PROJECT_ROOT_DIR, "Data")
# Criando o diretório, se ele não existir:
os.makedirs(PROJECT_DATA_PATH, exist_ok=True)

# Path: onde estão armazenadas as classes e funções que serão utilizadas neste notebook:
PROJECT_LIB_PATH = os.path.join(PROJECT_ROOT_DIR, "Library")

# Adicionando o diretório ao 'path' do Sistema, para podermos importar classes e funções que serão
# utilizadas neste notebook:
sys.path.append(PROJECT_LIB_PATH)

# Importando para este notebook, as classes e funções definidas no módulo "DataScience_Library_v1_0":
import DataScience_Library_v1_0 as dslib

# Importando para este notebook, as classes e funções definidas no módulo "DSA_Project_03_Library":
import DSA_Project_03_Library as pjlib

# Criando um objeto para calularmos os tempos de processamento:
et = dslib.ElapsedTime(builder_msg=False)

print("Setup Complete!")

Setup Complete!


In [2]:
# Versões dos pacotes usados neste jupyter notebook:
%reload_ext watermark
%watermark -a "Walter Trevisan" --iversions

numpy  1.18.2
pandas 0.25.3
Walter Trevisan


## `01-` Carregar o *dataset* de `testes`: `santander_test.pickle`

In [3]:
# Carregando o dataset "test.csv" que contém as variáveis preditoras:
test_df = pjlib.load_test_dataset_kaggle()

Loading file: ..\Datasets\test.csv
The label "TARGET" has been replaced by the label "Satisfaction"
Elapsed time: 3.11 seconds.


In [4]:
# Renomeando o label "ind_var10cte_ult1", em nosso dataset, para colocá-lo no mesmo padrão das outras variáveis (features).
test_df.rename(columns={'ind_var10cte_ult1': 'ind_var10_cte_ult1'}, inplace=True)

In [5]:
# Criando uma cópia do atributo "ID" que será utilizado na criação do arquivo que será submetido na competição:
output_df = test_df[['ID']].copy()

## `02-` Carregar o `objeto` criado na fase `03-Explore-The-Data (Step-05)`:

Agora, vamos carregar o seguinte **`objeto`**, criado na fase de análise exploratória dos dados: `03-Explore-The-Data (Step-05)`:

1. **`santander_relevant_feat`:** dicionário com as principais informações sobre cada uma das features **`Relevantes`**, ou seja, `tipo de variável`, valores ausentes (`NaN`), valores extremos (`Outliers`), e também uma lista com as variáveis que são **`Irrelevantes`**, ou seja, que não serão analisadas para a criação dos modelos preditivos (`Irrelevant`).

In [6]:
# 1. Carregando o objeto "santander_relevant_feat":
info = "The 'santander_relevant_feat' object has been successfully uploaded!"
dataset_relevant_feat = dslib.pickle_object_load(path=PROJECT_OBJ_PATH, file="santander_relevant_feat.pickle",
                                                 msg=info)

The 'santander_relevant_feat' object has been successfully uploaded!


## `03-` Carregar os `objetos` criados na fase de `04-Preprocessing`:

Vamos carregar os seguintes **`objetos`**, criados na fase de preparação dos dados (`04-Preprocessing`):

1. **`santander_feat_eng`:** objeto (dicionário) com informações sobre cada uma das transformações (**`Feature Engineering`**) que foram realizadas nas features **`Relevantes`** do nosso *data frame* de treinamento;

2. **`ohe`:** instância (objeto) criado da classe **`OneHotEncoder()`** onde foram realizadas transformações (**`Feature Engineering`**) nas features **`Categóricas`** do nosso *data frame* de treinamento;

3. **`pt`:** instância (objeto) criado da classe **`PowerTransformer()`** onde foram realizadas transformações (**`Feature Engineering`**) nas features **`Numéricas`** do nosso *data frame* de treinamento;

4. **`qt`:** instância (objeto) criado da classe **`QuantileTransformer()`** onde foram realizadas transformações (**`Feature Engineering`**) nas features **`Numéricas`** do nosso *data frame* de treinamento.

In [7]:
# 1. Carregando o objeto "santander_feat_eng":
dataset_feat_eng = dslib.pickle_object_load (path=PROJECT_OBJ_PATH, file="santander_feat_eng.pickle",
                                             msg="The 'santander_feat_eng' object has been successfully uploaded!")

# 2. Carregando o objeto "ohe":
ohe = dslib.pickle_object_load (path=PROJECT_OBJ_PATH, file="OneHotEncoder.pickle",
                                msg="The 'ohe' object has been successfully uploaded!")

# 3. Carregando o objeto "pt":
pt = dslib.pickle_object_load (path=PROJECT_OBJ_PATH, file="PowerTransformer.pickle",
                               msg="The 'pt' object has been successfully uploaded!")

# 4. Carregando o objeto "qt":
qt = dslib.pickle_object_load (path=PROJECT_OBJ_PATH, file="QuantileTransformer.pickle",
                               msg="The 'qt' object has been successfully uploaded!")

The 'santander_feat_eng' object has been successfully uploaded!
The 'ohe' object has been successfully uploaded!
The 'pt' object has been successfully uploaded!
The 'qt' object has been successfully uploaded!


## `04-` Carregar os `objetos` importantes (`Machine Learning`):

Vamos carregar os seguintes **`objetos`**, criados na fase **`05-Machine Learning`**:

### `4.1-` Objeto que contém as `features` mais importantes (`Feature Selection`) do modelo preditivo:

**`feat_sel_xgbc`**: Contém as *features* (mais importantes) selecionadas para o modelo preditivo treinado com o **`XGBC`**.

In [8]:
# Carregando as *features* (mais importantes) selecionadas para o modelo preditivo treinado com o "XGBC":
feat_sel_xgbc = dslib.pickle_object_load(path=PROJECT_ML_PATH, file="rfecv_xgbc_v1.pickle",
                                         msg="The features selection for 'XGBC' model has been successfully uploaded!")

The features selection for 'XGBC' model has been successfully uploaded!


### `4.2-` Objeto que contém o `modelo preditivo` treinado com os melhores `hiperparâmetros`:

**`xgbc`**: Modelo preditivo do algorito **`XGBC`** com os melhores hiperparâmetros selecionados.

In [9]:
# Carregando o modelo "XGBC" com os melhores hiperparâmetros selecionados:
xgbc = dslib.pickle_object_load(path=PROJECT_ML_PATH, file="rnd_search_xgbc_v2.pickle",
                                msg="The best 'XGBC' model has been successfully uploaded!")

The best 'XGBC' model has been successfully uploaded!


## `05-` Preparar o *data frame* de `testes`: **`test_df`**

Agora, vamos preparar o nosso *data frame* de *testes* executando as seguintes etapas:

> **`5.1-`** Remover do *data frame* as **`features`** irrelevantes (**`Irrelevant`**);


> **`5.2-`** Aplicar a função `OneHotEncoder()` nas features `Categóricas` (Qualitativas) do *data frame*;


> **`5.3-`** Aplicar a função `QuantileTransformer()` nas features `numéricas` (Quantitativas) do *data frame*;


> **`5.4-`** Aplicando a função `PowerTransformer()` nas features `numéricas` (Quantitativas) do *data frame*;


> **`5.5-`** Atualizar o *data frame* de `testes` com as features numéricas *transformadas*.

### `5.1-` Remover do *data frame* as `features` classificadas como `Irrelevant` (Irrelevantes):

In [10]:
# Resumo das features "relevantes" por tipo de informação:
print("Relevant features:")
print('     Binary: {}'.format(dataset_relevant_feat.count(keys=['Binary'])))
print('    Ordinal: {}'.format(dataset_relevant_feat.count(keys=['Ordinal'])))
print('Categorical: {}'.format(dataset_relevant_feat.count(keys=['Categorical'])))
print('   Discrete: {}'.format(dataset_relevant_feat.count(keys=['Discrete'])))
print(' Continuous: {}'.format(dataset_relevant_feat.count(keys=['Continuous'])))
print('----------------')
print('      Total: {}'.format(dataset_relevant_feat.count(keys=['Categorical', 'Binary', 'Ordinal', 'Discrete',
                                                                  'Continuous'])))

Relevant features:
     Binary: 1
    Ordinal: 0
Categorical: 4
   Discrete: 10
 Continuous: 8
----------------
      Total: 23


In [11]:
# Resumo das features "Irrelevantes":
print("Irrelevant features: {}".format(dataset_relevant_feat.count(keys=['Irrelevant'])))

Irrelevant features: 346


In [12]:
# Eliminando as features "Irrelevantes" do data frame de testes:
test_df.drop(columns=dataset_relevant_feat.get(key='Irrelevant'), inplace=True)

# Eliminando a feature "ID" (Primary Key) do data frame de testes:
test_df.drop(columns='ID', inplace=True)

# Resumo do data frame:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75818 entries, 0 to 75817
Data columns (total 23 columns):
var15                       75818 non-null int64
ind_var30                   75818 non-null int64
num_var4                    75818 non-null int64
num_var5                    75818 non-null int64
num_var30                   75818 non-null int64
num_var35                   75818 non-null int64
num_var42                   75818 non-null int64
saldo_var5                  75818 non-null float64
saldo_var30                 75818 non-null float64
saldo_var42                 75818 non-null float64
var36                       75818 non-null int64
num_var22_ult3              75818 non-null int64
num_meses_var5_ult3         75818 non-null int64
num_meses_var39_vig_ult3    75818 non-null int64
num_var45_hace2             75818 non-null int64
num_var45_hace3             75818 non-null int64
num_var45_ult1              75818 non-null int64
num_var45_ult3              75818 non-null int64
sal

### `5.2-` Aplicar a função `OneHotEncoder()` nas features `Categóricas` (Qualitativas) do *data frame*:

In [13]:
# Obtem todas as variáveis preditoras (features) categóricas (Relevantes):
category_feats = dataset_relevant_feat.get(key='Binary') + dataset_relevant_feat.get(key='Categorical')
print("Category features: {}".format(category_feats))

Category features: ['ind_var30', 'num_var4', 'var36', 'num_meses_var5_ult3', 'num_meses_var39_vig_ult3']


In [14]:
# Obtendo o número único de entradas em cada feature com dados categóricos no data frame de testes:
test_category_num_unique = list(map(lambda feat: test_df[feat].nunique(), category_feats))
d = dict(zip(category_feats, test_category_num_unique))

# Imprime as categorias em ordem crescente pela quantidde de classes únicas:
sorted(d.items(), key=lambda x: x[1])

[('ind_var30', 2),
 ('num_meses_var5_ult3', 4),
 ('num_meses_var39_vig_ult3', 4),
 ('var36', 5),
 ('num_var4', 10)]

In [15]:
# Obtendo as entradas únicas em cada feature com dados categóricos no data frame de testes:
test_category_unique = list(map(lambda feat: test_df[feat].unique(), category_feats))
test_cat_unique_dict = dict(zip(category_feats, test_category_unique))

# Mostrando as features categóricas com as suas categorias:
test_cat_unique_dict

{'ind_var30': array([1, 0], dtype=int64),
 'num_var4': array([1, 2, 0, 6, 3, 4, 5, 7, 8, 9], dtype=int64),
 'var36': array([ 3, 99,  1,  2,  0], dtype=int64),
 'num_meses_var5_ult3': array([3, 0, 2, 1], dtype=int64),
 'num_meses_var39_vig_ult3': array([2, 0, 1, 3], dtype=int64)}

In [16]:
# Agora, vamos verificar se todas as categorias no data frame de "testes" estão contidas no data frame de "treinamento":
train_cat_unique_dict = dict(zip(category_feats, ohe.categories_))
train_cat_unique_dict

{'ind_var30': array([0, 1], dtype=int64),
 'num_var4': array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int64),
 'var36': array([ 0,  1,  2,  3, 99], dtype=int64),
 'num_meses_var5_ult3': array([0, 1, 2, 3], dtype=int64),
 'num_meses_var39_vig_ult3': array([0, 1, 2, 3], dtype=int64)}

**Análise:** podemos observar na feature **`num_var4`**, que duas categorias (**`8` e `9 `**) no *data frame* de testes não estão contidas no *data frame* de treinamento. Ou seja, ao aplicarmos a função **`OneHotEncoder()`** em nosso *data frame* de testes, não teremos duas colunas de dados relacionados a estas duas categorias. Se estas duas categorias forem significativas (importantes) para a classificação dos clientes, o nosso modelo muito provavelmente apresentará um resultado (**`score`**) ruim para a métrica **`AUC`**.

In [17]:
# Criando uma cópia do dataset de testes que contém apenas os atributos categóricos:
test_cat = test_df[category_feats].copy()

# Aplicando o "one-hot encoding" nos dados de teste:
test_cat_ohe = ohe.transform(X=test_cat.values).toarray()

# Definindo os nomes das features:
cat_labels = [feat + '_cat' for feat in category_feats]
feat_names = list(ohe.get_feature_names(cat_labels))

# Criando um data frame com as features "OHE":
test_cat_ohe_df = pd.DataFrame(data=test_cat_ohe, columns=feat_names)

# Incluindo as features "OHE" em nosso data frame de testes:
test_df = pd.concat([test_df, test_cat_ohe_df], axis=1)

# Excluindo as "features" categóricas do nosso data frame de testes, porque as mesmas já foram transformadas (OHE):
test_df.drop(columns=category_feats, inplace=True)

# Resumo do nosso data frame de testes:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75818 entries, 0 to 75817
Data columns (total 41 columns):
var15                             75818 non-null int64
num_var5                          75818 non-null int64
num_var30                         75818 non-null int64
num_var35                         75818 non-null int64
num_var42                         75818 non-null int64
saldo_var5                        75818 non-null float64
saldo_var30                       75818 non-null float64
saldo_var42                       75818 non-null float64
num_var22_ult3                    75818 non-null int64
num_var45_hace2                   75818 non-null int64
num_var45_hace3                   75818 non-null int64
num_var45_ult1                    75818 non-null int64
num_var45_ult3                    75818 non-null int64
saldo_medio_var5_hace2            75818 non-null float64
saldo_medio_var5_hace3            75818 non-null float64
saldo_medio_var5_ult1             75818 non-null float64

### `5.3-` Aplicar a função `QuantileTransformer()` nas features `numéricas` (Quantitativas) do *data frame*:

In [18]:
# Visializando todas as "features" onde aplicaremos a função "QuantileTransformer()":
print("Feature Engineering (Quantile Transformation): {}".format(len(dataset_feat_eng["QuantileTransformation"])))
print(dataset_feat_eng["QuantileTransformation"])

Feature Engineering (Quantile Transformation): 1
['var38']


In [19]:
# Criando uma cópia das features numéricas onde aplicaremos a função "QuantileTransformer()":
num_feat_df = test_df[dataset_feat_eng["QuantileTransformation"]].copy()

# Padronizando as features com a função:
qt_num_feat = qt.transform(num_feat_df)

# Criando um novo "data frame" com as features transformadas:
cols = [feat + "_qt" for feat in num_feat_df.columns]
qt_num_feat_df = pd.DataFrame(qt_num_feat, columns=cols)

### `5.4-` Aplicando a função `PowerTransformer()` nas features `numéricas` (Quantitativas) do *data frame*:

In [20]:
# Visializando todas as "features" onde aplicaremos a função "PowerTransformer()":
print("Feature Engineering (Power Transformation): {}".format(len(dataset_feat_eng["PowerTransformation"])))
print(dataset_feat_eng["PowerTransformation"])

Feature Engineering (Power Transformation): 17
['var15', 'num_var35', 'num_var42', 'num_var22_ult3', 'num_var30', 'num_var45_hace2', 'num_var45_hace3', 'num_var45_ult1', 'num_var45_ult3', 'num_var5', 'saldo_var42', 'saldo_var30', 'saldo_medio_var5_hace2', 'saldo_medio_var5_hace3', 'saldo_medio_var5_ult1', 'saldo_medio_var5_ult3', 'saldo_var5']


In [21]:
# Criando uma cópia das features numéricas onde aplicaremos a função "PowerTransformer()":
num_feat_df = test_df[dataset_feat_eng["PowerTransformation"]].copy()

# Padronizando as features com a função:
pt_num_feat = pt.transform(num_feat_df)

# Criando um novo "data frame" com as features transformadas:
cols = [feat + "_pt" for feat in num_feat_df.columns]
pt_num_feat_df = pd.DataFrame(pt_num_feat, columns=cols)

### `5.5-` Atualizar o *data frame* de `testes` com as features numéricas *transformadas*:

In [22]:
# Excluindo as "features" "Quantitativas Discretas e Contínuas" do nosso data frame de testes:
num_feats = dataset_relevant_feat.get(key='Discrete') + dataset_relevant_feat.get(key='Continuous')
test_df.drop(columns=num_feats, inplace=True)

# Incluindo as features "Quantitativas Discretas e Contínuas" ("transformadas") em nosso data frame de testes:
test_df = pd.concat([test_df, qt_num_feat_df, pt_num_feat_df], axis=1)

# Resumo do nosso data frame de testes:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75818 entries, 0 to 75817
Data columns (total 41 columns):
ind_var30_cat_0                   75818 non-null float64
ind_var30_cat_1                   75818 non-null float64
num_var4_cat_0                    75818 non-null float64
num_var4_cat_1                    75818 non-null float64
num_var4_cat_2                    75818 non-null float64
num_var4_cat_3                    75818 non-null float64
num_var4_cat_4                    75818 non-null float64
num_var4_cat_5                    75818 non-null float64
num_var4_cat_6                    75818 non-null float64
num_var4_cat_7                    75818 non-null float64
var36_cat_0                       75818 non-null float64
var36_cat_1                       75818 non-null float64
var36_cat_2                       75818 non-null float64
var36_cat_3                       75818 non-null float64
var36_cat_99                      75818 non-null float64
num_meses_var5_ult3_cat_0         75

In [23]:
# Criando um array com os dados das "features" no objeto "X_test":
X_test = test_df.values
X_test.shape

(75818, 41)

## `06-` Realizar os `testes` com o nosso melhor `modelo preditivo (ML)`:

## [XGBC](https://xgboost.readthedocs.io/en/latest/python/python_api.html?highlight=xgbclassifier#xgboost.XGBClassifier) (XGBoost Classifier)

### `Feature Selection`:

Vamos utilizar as *features* selecionadas (**`importantes`**), que foram definidas na fase anterior (**`05-Machine Learning`**).

In [24]:
# Selecionando os dados de treinamento, de acordo com as features selecionadas (Importantes) na fase anterior (ML):
X_test_xgbc = X_test[:, feat_sel_xgbc.get_support()]
print("X_test_xgbc (shape):",X_test_xgbc.shape)

X_test_xgbc (shape): (75818, 32)


### `Predict`

In [25]:
# Calculando as previsões nos dados de teste:
# Verifica se vamos fazer as previsões:
if (PREDICT): # Utilizar "PREDICT=True" (no Setup) para fazer as 'previsões'!
    # Inicia as previsões:
    et.start(et.start(msg="Starting predict (XGBC)..."))
    y_test_pred_xgbc = xgbc.predict(X=X_test_xgbc)
    
    # Fim das previsões:
    et.end(msg="Time consumed for predict (XGBC):")
    
    # Salvando as previsões:
    dslib.pickle_object_save (path=PROJECT_OUTPUT_PATH, file="y_test_pred_xgbc.pickle", object_name=y_test_pred_xgbc,
                              msg="The predicts (tests) with 'XGBC' model was successfully saved!")
else:
    # As previsões já foram feitas!
    # Carregando as previsões "y_test_pred_xgbc.pickle":
    y_test_pred_xgbc = dslib.pickle_object_load(path=PROJECT_OUTPUT_PATH, file="y_test_pred_xgbc.pickle",
                                                msg="The predicts (tests) with 'XGBC' model has been successfully uploaded!")

Starting predict (XGBC)...
Time consumed for predict (XGBC): 0.45 seconds.
The predicts (tests) with 'XGBC' model was successfully saved!


## `07-` Criar e salvar o arquivo que será submetido na plataforma `Kaggle`:

### `sample_submission.csv`

In [26]:
# Armazenando os resultados no data frame de saída "output_df":
output_df['TARGET'] = y_test_pred_xgbc

# Definindo o 'ID' como índice:
output_df.set_index(keys='ID', inplace=True)

# Visualizando os resultados:
output_df

,TARGET
ID,
2,0
5,0
6,0
7,0
9,0
...,...
151831,0
151832,0
151833,0


In [27]:
# Definindo o 'Path' para salvar o arquivo:
sample_submission_file = os.path.join(PROJECT_OUTPUT_PATH, "sample_submission.csv")

# Salvando os resultados:
output_df.to_csv(sample_submission_file)

## `08-` Verificar o `score` e a posição no `leaderboard`, na plataforma `Kaggle`:

### `Score`:

![Kaggle-Score.png](.\output\Kaggle-Score.png)

### `Leaderboard`:

![Kaggle-Leaderboard.png](.\output\Kaggle-Leaderboard.png)

## <font color='black'>FIM</font>